In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
import io
dataset = pd.read_csv('../input/car-price-prediction/CarPrice_Assignment.csv')
# Dataset is now stored in a Pandas Dataframe

In [ ]:
dataset = dataset.sample(frac=1)

In [ ]:
dataset.head()

In [ ]:
dataset.info()

In [ ]:
dataset.isnull().sum()

In [ ]:
dataset.isna().sum()

In [ ]:
dataset.describe()

Drop car_ID and CarName

In [ ]:
dataset.drop(columns='car_ID', inplace=True)
dataset.drop(columns='CarName', inplace=True)
dataset.head(10)


One hot encoding

In [ ]:
dataset = pd.get_dummies(dataset, prefix=['fueltype','aspiration', 'doornumber','carbody', 'drivewheel', 'enginelocation', 'enginetype','cylindernumber', 
                                          'fuelsystem'])

In [ ]:
dataset.head()

In [ ]:
dataset.columns

Split train and test set

In [ ]:
y = dataset['price']

In [ ]:
y

In [ ]:
dataset.drop(columns='price', inplace=True)

In [ ]:
dataset.head()

In [ ]:
dataset.shape

In [ ]:
X_train = dataset[:104]

In [ ]:
Y_train = y[:104]

In [ ]:
X_test = dataset[104:165]

In [ ]:
Y_test = y[104:165]

In [ ]:
X_val = dataset[165:]

In [ ]:
Y_val = y[165:]

Normalization of data

In [ ]:
X_train

In [ ]:
mean = X_train.iloc[: , 0:14].mean(axis=0)
X_train.iloc[: , 0:14] -= mean
std = X_train.iloc[:, 0:14].std(axis=0)
X_train.iloc[: , 0:14] /= std 

In [ ]:
X_train

In [ ]:
val_mean = X_val.iloc[:, 0:14].mean(axis=0)
val_std = X_val.iloc[:, 0:14].std(axis=0)
X_val.iloc[:, 0:14] -= val_mean
X_val.iloc[:, 0:14] /= val_std

In [ ]:
test_mean = X_test.iloc[:, 0:14].mean(axis=0)
test_std =  X_test.iloc[:, 0:14].std(axis=0)
X_test.iloc[:, 0:14] -= test_mean
X_test.iloc[:, 0:14] /= test_std
X_test

In [ ]:
X_train.shape[1]

In [ ]:
X_val.shape

Build a Model

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras import regularizers

In [ ]:
network = Sequential()

In [ ]:
network.add(layers.Dense(16, activation='relu', kernel_regularizer =regularizers.l2(0.02),   input_shape=(X_train.shape[1],)))
network.add(layers.Dense(14, activation='relu', kernel_regularizer =regularizers.l2(0.02)))
network.add(layers.Dense(8, activation='relu', kernel_regularizer =regularizers.l2(0.02)))
network.add(layers.Dense(6, activation='relu', kernel_regularizer =regularizers.l2(0.002)))
# network.add(layers.Dense(4, activation='relu', kernel_regularizer =regularizers.l2(0.002)))
network.add(layers.Dense(1))

In [ ]:
network.compile(optimizer='rmsprop', loss='mse', metrics='mae')

In [ ]:
history = network.fit(X_train,Y_train, batch_size=16, verbose=0, epochs=200, validation_data=(X_val, Y_val))

In [ ]:
epochs = range(1, 201)

In [ ]:
train_mae = history.history['mae']
val_mae = history.history['val_mae']

In [ ]:
plt.plot(epochs, train_mae, 'r', label='Training MAE')
plt.plot(epochs, val_mae, 'g', label='Validation MAE')
plt.title('Training and Validation loss ')
plt.xlabel('Epochs')
plt.ylabel('MAE')
plt.legend()
plt.show()

In [ ]:
train_loss = history.history['loss']
val_loss = history.history['val_loss']

In [ ]:
plt.plot(epochs, train_loss, 'r', label='Training Loss')
plt.plot(epochs, val_loss, 'g', label='Validation Loss')
plt.title('Training and Validation loss ')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
Y_test

In [ ]:
network.predict(X_test)

In [ ]:
test_mse_score, test_mae_score  =network.evaluate(X_test, Y_test)

<h2> Let's try with K-Fold technique </h2>

In [ ]:
frames = [X_train,X_val ]
X_train_k_fold = pd.concat(frames)

In [ ]:
X_train_k_fold.shape

In [ ]:
Y_frames = [Y_train, Y_val]
Y_trainKfold = pd.concat(Y_frames)

In [ ]:
Y_trainKfold

In [ ]:
from tensorflow.keras import regularizers
def build_model():
    model = Sequential()
    model.add(layers.Dense(16, activation='relu', kernel_regularizer=regularizers.l2(0.02), input_shape=(X_train_k_fold.shape[1],)))
    model.add(layers.Dense(12, activation='relu', kernel_regularizer=regularizers.l2(0.02)))
    model.add(layers.Dense(8, activation='relu', kernel_regularizer=regularizers.l2(0.02)))
    model.add(layers.Dense(4, activation='relu', kernel_regularizer=regularizers.l2(0.02)))
    model.add(layers.Dense(1))
    model.compile(optimizer='adam', loss='mse', metrics=['mae'])
    return model

In [ ]:
import numpy as np
k = 2
num_val_samples = len(X_train_k_fold) // k
num_epochs = 300
all_scores = []

In [ ]:

for i in range(k):
    print('processing fold #', i)
    val_data = X_train_k_fold[i * num_val_samples: (i + 1) * num_val_samples]
    val_targets = Y_trainKfold[i * num_val_samples: (i + 1) * num_val_samples]
    partial_train_data = np.concatenate([X_train_k_fold[:i * num_val_samples],
                                         X_train_k_fold[(i + 1) * num_val_samples:]], axis=0)
    partial_train_targets = np.concatenate([Y_trainKfold[:i * num_val_samples],
                                            Y_trainKfold[(i + 1) * num_val_samples:]],
                                           axis=0)
    model = build_model()
    history = model.fit(partial_train_data, partial_train_targets,
    epochs=num_epochs, batch_size=30,verbose=0, validation_data=(val_data, val_targets))

    k_Fold_train_loss = history.history['mae']
    k_Fold_val_loss = history.history['val_mae']
    epochs = range(1,301)
    plt.plot(epochs, k_Fold_train_loss, 'r', label='Training MAE')
    plt.plot(epochs, k_Fold_val_loss, 'g', label='Validation MAE')
    plt.title('Training and Validation MAE ')
    plt.xlabel(' Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.show()
    print('Val_MAE:   '+str(k_Fold_val_loss[-1]))
    print('Predicted Prices:    '+str(model.predict(X_test)))
    

<center> <h1> Thanks ❤ </h1></center>